# Assignment 2 Checkpoint

```
- Machine Learning, Innopolis University (Fall semester 2021)
- Professor: Adil Khan
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>

```
Lab Plan 
1. Autoencoders 
2. Custom loss functions
3. Data Preprocessing
```

<hr>

## 1. Autoencoders 

* Types of autoencoders
* Applications of autoencoders
* Autoencoders training procedure
* Reparametrisation trick

![caption](https://miro.medium.com/max/2400/1*Q5dogodt3wzKKktE0v3dMQ@2x.png)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## 1.1 Defining AE

In [ ]:
class autoencoder(nn.Module):
    def __init__(self, input_size, latent_dim):
        super(autoencoder, self).__init__()
        # Step 1 : Define the encoder 
        # Step 2 : Define the decoder
        # Step 3 : Initialize the weights (optional)
        self.encoder = nn.Sequential(
          nn.Linear(input_size, input_size//2),
          nn.ReLU(True),
          nn.Linear(input_size//2, input_size//3),
          nn.Linear(input_size//3, input_size//4),
          nn.Tanh(),
          nn.Linear(input_size//4, latent_dim)
        )
        self.decoder = nn.Sequential(
          nn.Linear(latent_dim, input_size//4),
          nn.ReLU(True),
          nn.Linear(input_size//4, input_size//3),
          nn.Linear(input_size//3, input_size//2),
          nn.Tanh(),
          nn.Linear(input_size//2, input_size)
        )
        self.encoder.apply(self.__init_weights)
        self.decoder.apply(self.__init_weights)
        
    def forward(self, x):
        # Step 1: Pass the input through encoder to get latent representation
        # Step 2: Take latent representation and pass through decoder
        x = self.encoder(x)
        x = self.decoder(x)
        return x



    def encode(self,input):
        #Step 1: Pass the input through the encoder to get latent representation
        return self.encoder(input)

    def __init_weights(self,m):
        #Init the weights (optional)
        if type(m) == nn.Linear:
            torch.nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(0.01)

## 1.2 Training AE

In [ ]:
batchSize = 100
learning_rate = 0.01
num_epochs = 5
sample = torch.randn((batchSize,1,64))
AE = autoencoder(64,5).to(device)
print(AE)
# print(summary(AE,input_size=(1, 64)))

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(AE.parameters(),lr=learning_rate)

#Create a random dataset
data_loader = DataLoader(TensorDataset(torch.randn((1000,1,64))),batch_size=32,shuffle=True)

for epoch in range(num_epochs):
    epoch_loss = 0.0
    for X in data_loader:
        X = X[0].to(device)

        optimizer.zero_grad()
        # forward
        output = AE(X)
        loss = criterion(output, X)

        # backward
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # log
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))

## 1.3 CNN AE

Load MNIST Dataset

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 1 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 1, 2, stride=2)


    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)  # compressed representation
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x

# initialize the NN
model = ConvAutoencoder()
print(model)